In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# Initialize lists and URL
movie_names = []
years = []
ratings = []
metascores = []
gross_incomes = []
votes = []
runtimes = []
genres = []
certificates = []
descriptions = []
directors = []
stars = []

In [3]:
url = 'https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=release_date,desc'


In [4]:
count = 0
page_number = 1
while count < 10000:
    page_url = f'{url}&start={50 * (page_number - 1) + 1}'
    response = requests.get(page_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    movie_data = soup.findAll('div', attrs={'class': 'lister-item-content'})

    for store in movie_data:
        year_span = store.h3.find('span', class_='lister-item-year')
        if year_span:
            year_text = year_span.text.strip('()')
            if len(year_text) > 4:  # Check for any unwanted text in the year string
                year_of_release = year_text[-4:]
            else:
                year_of_release = year_text
            try:
                if 2019 <= int(year_of_release) <= 2023:
                    # Extract movie name
                    name = store.h3.a.text
                    movie_names.append(name)

                    # Extract year of release
                    years.append(year_of_release)

                    # Extract runtime (with error handling)
                    time = store.p.find('span', class_='runtime')
                    runtimes.append(time.text.replace(' min', '') if time else 'N/A')

                    # Extract IMDb rating
                    rate = store.find('div', class_='inline-block ratings-imdb-rating').strong.text
                    ratings.append(rate)

                    # Extract Metascore (if available)
                    meta = store.find('span', class_='metascore')
                    metascores.append(meta.text.strip() if meta else 'NAN')

                    # Extract number of votes
                    vote = store.find('span', {'name': 'nv'})['data-value']
                    votes.append(vote)

                    # Extract gross income
                    gross = store.find_all('span', {'name': 'nv'})[-1]['data-value']
                    gross_incomes.append(gross if gross else 'NAN')

                    # Extract genre
                    genre = store.find('span', class_='genre').text.strip()
                    genres.append(genre)

                    # Extract certificate
                    certificate = store.find('span', class_='certificate').text if store.find('span', class_='certificate') else 'N/A'
                    certificates.append(certificate)

                    # Extract description
                    description = store.find_all('p', class_='text-muted')[1].text.strip() if len(store.find_all('p', class_='text-muted')) > 1 else 'N/A'
                    descriptions.append(description)

                    # Extracting directors and stars
                    credits = store.find_all('p')[2].text.split('|')
                    director = credits[0].strip().replace('Director:', '')
                    directors.append(director)

                    stars_info = credits[1].strip().replace('Stars:', '') if len(credits) > 1 else 'N/A'
                    stars_list = stars_info.split(',')
                    stars.append(', '.join([star.strip() for star in stars_list]))

                    count += 1
                    if count >= 10000:
                        break
            except ValueError as e:
                print(f"Error: {e} - Could not convert {year_of_release} to an integer.")

    page_number += 1



In [5]:
# Print the extracted movie data
for i in range(len(movie_names)):
    print(f"Movie: {movie_names[i]}")
    print(f"Year: {years[i]}")
    print(f"Rating: {ratings[i]}")
    print(f"Metascore: {metascores[i]}")
    print(f"Gross Income: {gross_incomes[i]}")
    print(f"Votes: {votes[i]}")
    print(f"Runtime: {runtimes[i]}")
    print(f"Genre: {genres[i]}")
    print(f"Certificate: {certificates[i]}")
    print(f"Description: {descriptions[i]}")
    print(f"Director: {directors[i]}")
    print(f"Stars: {stars[i]}")
    print("-----------------------\n")

# Now, you have extracted 10,000 movies from 2019 to 2023.


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
MOVIE_data = pd.DataFrame({'Name of movie': movie_names, 'YEAR': years, 'RATES': ratings, 'METASCORE': metascores,
                            'gross_collection': gross_incomes, 'Votes': votes, 'Watch Time': runtimes,
                            'Genre': genres, 'Certificate': certificates, 'Description': descriptions,
                            'Director': directors, 'Stars': stars})

In [9]:
MOVIE_data

,Name of movie,YEAR,RATES,METASCORE,gross_collection,Votes,Watch Time,Genre,Certificate,Description,Director,Stars
0,Nowhere,2023,6.4,NAN,15948,15948,109,"Drama, Thriller",N/A,A young pregnant woman named Mia escapes from ...,\nAlbert Pintó,"Anna Castillo, Tamar Novas, Tony Corvillo, Mar..."
1,Totally Killer,2023,6.6,62,14778,14778,106,"Comedy, Horror",R,"When the infamous ""Sweet Sixteen Killer"" retur...",\nNahnatchka Khan,"Kiernan Shipka, Olivia Holt, Charlie Gillespie..."
2,Saw X,2023,7.1,60,18028,18028,118,"Horror, Mystery, Thriller",R,A sick and desperate John travels to Mexico fo...,\nKevin Greutert,"Tobin Bell, Shawnee Smith, Synnøve Macody Lund..."
3,Fukrey 3,2023,7.7,NAN,20316,20316,147,"Comedy, Crime, Drama",N/A,The friends facing disastrously funny situatio...,\nMrighdeep Lamba,"Pulkit Samrat, Varun Sharma, Manjot Singh, Ric..."
4,No One Will Save You,2023,6.3,60,37397,37397,93,"Horror, Sci-Fi, Thriller",PG-13,An exiled anxiety-ridden homebody must battle ...,\nBrian Duffield,"Kaitlyn Dever, Elizabeth Kaluev, Zack Duhame, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Sound of Freedom,2023,7.8,36,70625,70625,131,"Biography, Crime, Drama",PG-13,The incredible true story of a former governme...,\nAlejandro Monteverde,"Jim Caviezel, Mira Sorvino, Bill Camp, Cristal..."
9996,Satyaprem Ki Katha,2023,7.2,NAN,27667,27667,146,"Comedy, Drama, Musical",UA,"A middle-class boy in Ahmedabad, Satyaprem fal...",\nSameer Vidwans,"Kartik Aaryan, Kiara Advani, Supriya Pathak, G..."
9997,Lust Stories 2,2023,6.5,NAN,50562,50562,132,"Drama, Romance",A,"Four eminent Indian directors explore sex, des...","Directors:\nR. Balki, \nSujoy Ghosh, \nAmit Ra...","Tamannaah Bhatia, Vijay Varma, Neena Gupta, Mr..."
9998,Carry on Jatta 3,2023,6.8,NAN,10913,10913,136,"Comedy, Drama, Romance",UA,"Advocate Dhillon's son, Jass, is in love with ...",\nSmeep Kang,"Gippy Grewal, Sonam Bajwa, Binnu Dhillon, Rupi..."


In [10]:
MOVIE_data.to_csv("imdb_10000_movies1.csv", index=False)